## GEE Asset Exporter

This script exports monthly Landsat-derived DSWE products (DSWE mask, QC mask, and RGB of composite) and monthly Sentinel-2 based DSWE products. The code first downloads assets within the given GEE asset folder. Then the code downloads all .tifs of the given asset type and compiles then within the given directory.

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: October 1st, 2025

## Import packages

In [1]:
import ee
import re
import datetime
import os

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

ee.Initialize()

## Export GEE assets to Drive

In [9]:
def export_gee_assets(
    asset_root_path,
    drive_folder,
    start_year,
    end_year,
    bands='ALL',  # 'ALL' or list of bands like ['red', 'green', 'blue']
    filename_pattern=r'(\d{4})_(\d{2})',
    filename_prefix='Export',
    scale=30,
    crs='EPSG:4326'
):
    """
    Flexible Earth Engine asset exporter to Google Drive with multi-band support.

    Parameters:
        asset_root_path (str): GEE folder containing assets
        drive_folder (str): Google Drive folder to export to
        start_year (int): Minimum year to include
        end_year (int): Maximum year to include
        bands (list or 'ALL'): Band names to export or 'ALL' for all bands
        filename_pattern (str): Regex pattern to extract year/month
        filename_prefix (str): Optional prefix for export filename
        scale (int): Pixel scale in meters
        crs (str): Coordinate reference system
    """
    asset_list = ee.data.listAssets({'parent': asset_root_path})['assets']

    for asset in asset_list:
        asset_id = asset['name']
        basename = asset_id.split('/')[-1]

        # Extract year and month with custom regex
        match = re.search(filename_pattern, basename)
        if not match:
            print(f"⚠️ Skipping malformed asset name: {basename}")
            continue

        year = int(match.group(1))
        month = int(match.group(2))

        if not (start_year <= year <= end_year):
            print(f"❌ Skipping {basename}: outside year range")
            continue

        print(f"✅ Preparing export for: {basename}")
        image = ee.Image(asset_id)

        # Select bands if specified
        if bands != 'ALL':
            image = image.select(bands)

        # Use the image's geometry
        region = image.geometry()

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{filename_prefix}_{basename}",
            folder=drive_folder,
            fileNamePrefix=basename,
            region=region.getInfo()['coordinates'],
            scale=scale,
            crs=crs,
            maxPixels=1e13
        )
        task.start()

In [13]:
# ✅ Example Usage: Export RGB + DSWE bands
export_gee_assets(
    asset_root_path="projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products",
    drive_folder="Sentinel2_DSWE",
    start_year=2018,
    end_year=2025,
    bands=['DSWE'], 
    filename_pattern=r'DSWE_(\d{4})_(\d{2})',
    filename_prefix="Sentinel2",
    scale = 10
)

✅ Preparing export for: DSWE_2018_12
✅ Preparing export for: DSWE_2019_01
✅ Preparing export for: DSWE_2019_02
✅ Preparing export for: DSWE_2019_03
✅ Preparing export for: DSWE_2019_04
✅ Preparing export for: DSWE_2019_05
✅ Preparing export for: DSWE_2019_06
✅ Preparing export for: DSWE_2019_07
✅ Preparing export for: DSWE_2019_08
✅ Preparing export for: DSWE_2019_09
✅ Preparing export for: DSWE_2019_10
✅ Preparing export for: DSWE_2019_11
✅ Preparing export for: DSWE_2019_12
✅ Preparing export for: DSWE_2020_01
✅ Preparing export for: DSWE_2020_02
✅ Preparing export for: DSWE_2020_03
✅ Preparing export for: DSWE_2020_04
✅ Preparing export for: DSWE_2020_05
✅ Preparing export for: DSWE_2020_06
✅ Preparing export for: DSWE_2020_07
✅ Preparing export for: DSWE_2020_08
✅ Preparing export for: DSWE_2020_09
✅ Preparing export for: DSWE_2020_10
✅ Preparing export for: DSWE_2020_11
✅ Preparing export for: DSWE_2020_12
✅ Preparing export for: DSWE_2021_01
✅ Preparing export for: DSWE_2021_02
✅

## Monitor GEE tasks

In [2]:
def monitor_tasks():
    tasks = ee.batch.Task.list()

    if not tasks:
        print("⚠️ No tasks found.")
        return

    print(f"{'Description':<35} {'State':<12} {'Type':<10} {'Created'}")
    print("-" * 80)

    for task in tasks:
        state = task.status().get('state')
        task_type = task.task_type
        description = task.config.get('description', 'N/A')
        creation_time_ms = task.status().get('creation_timestamp_ms')

        if creation_time_ms:
            creation_time = datetime.datetime.fromtimestamp(creation_time_ms / 1000)
        else:
            creation_time = "N/A"

        print(f"{description:<35} {state:<12} {task_type:<10} {creation_time}")

# Example usage:
monitor_tasks()


Description                         State        Type       Created
--------------------------------------------------------------------------------
Sentinel2_DSWE_2025_08              READY        EXPORT_IMAGE 2025-10-06 16:02:17.038000
Sentinel2_DSWE_2025_07              READY        EXPORT_IMAGE 2025-10-06 16:02:16.754000
Sentinel2_DSWE_2025_06              READY        EXPORT_IMAGE 2025-10-06 16:02:16.330000
Sentinel2_DSWE_2025_05              READY        EXPORT_IMAGE 2025-10-06 16:02:16.028000
Sentinel2_DSWE_2025_04              READY        EXPORT_IMAGE 2025-10-06 16:02:15.666000
Sentinel2_DSWE_2025_03              READY        EXPORT_IMAGE 2025-10-06 16:02:15.283000
Sentinel2_DSWE_2025_02              READY        EXPORT_IMAGE 2025-10-06 16:02:14.906000
Sentinel2_DSWE_2025_01              READY        EXPORT_IMAGE 2025-10-06 16:02:14.571000
Sentinel2_DSWE_2024_12              READY        EXPORT_IMAGE 2025-10-06 16:02:14.042000
Sentinel2_DSWE_2024_11              READY        E

Landsat_Composite_2024_09           READY        EXPORT_IMAGE 2025-10-06 16:00:07.813000
Landsat_Composite_2024_08           READY        EXPORT_IMAGE 2025-10-06 16:00:07.447000
Landsat_Composite_2024_07           READY        EXPORT_IMAGE 2025-10-06 16:00:07.164000
Landsat_Composite_2024_06           READY        EXPORT_IMAGE 2025-10-06 16:00:06.817000
Landsat_Composite_2024_05           READY        EXPORT_IMAGE 2025-10-06 16:00:06.478000
Landsat_Composite_2024_04           READY        EXPORT_IMAGE 2025-10-06 16:00:06.109000
Landsat_Composite_2024_03           READY        EXPORT_IMAGE 2025-10-06 16:00:05.788000
Landsat_Composite_2024_02           READY        EXPORT_IMAGE 2025-10-06 16:00:05.503000
Landsat_Composite_2024_01           READY        EXPORT_IMAGE 2025-10-06 16:00:05.209000
Landsat_Composite_2023_12           READY        EXPORT_IMAGE 2025-10-06 16:00:04.851000
Landsat_Composite_2023_11           READY        EXPORT_IMAGE 2025-10-06 16:00:04.511000
Landsat_Composite_202

Landsat_Composite_2016_12           READY        EXPORT_IMAGE 2025-10-06 15:59:36.763000
Landsat_Composite_2016_11           READY        EXPORT_IMAGE 2025-10-06 15:59:36.416000
Landsat_Composite_2016_10           READY        EXPORT_IMAGE 2025-10-06 15:59:35.874000
Landsat_Composite_2016_09           READY        EXPORT_IMAGE 2025-10-06 15:59:35.500000
Landsat_Composite_2016_08           READY        EXPORT_IMAGE 2025-10-06 15:59:35.127000
Landsat_Composite_2016_07           READY        EXPORT_IMAGE 2025-10-06 15:59:34.761000
Landsat_Composite_2016_06           READY        EXPORT_IMAGE 2025-10-06 15:59:34.396000
Landsat_Composite_2016_05           READY        EXPORT_IMAGE 2025-10-06 15:59:34.062000
Landsat_Composite_2016_04           READY        EXPORT_IMAGE 2025-10-06 15:59:33.648000
Landsat_Composite_2016_03           READY        EXPORT_IMAGE 2025-10-06 15:59:33.334000
Landsat_Composite_2016_02           READY        EXPORT_IMAGE 2025-10-06 15:59:33.016000
Landsat_Composite_201

Landsat_Composite_2005_03           READY        EXPORT_IMAGE 2025-10-06 15:58:54.670000
Landsat_Composite_2005_02           READY        EXPORT_IMAGE 2025-10-06 15:58:54.397000
Landsat_Composite_2004_09           READY        EXPORT_IMAGE 2025-10-06 15:58:54.035000
Landsat_Composite_2004_08           READY        EXPORT_IMAGE 2025-10-06 15:58:53.755000
Landsat_Composite_2004_07           READY        EXPORT_IMAGE 2025-10-06 15:58:53.439000
Landsat_Composite_2004_06           READY        EXPORT_IMAGE 2025-10-06 15:58:52.942000
Landsat_Composite_2004_05           READY        EXPORT_IMAGE 2025-10-06 15:58:52.642000
Landsat_Composite_2004_04           READY        EXPORT_IMAGE 2025-10-06 15:58:52.324000
Landsat_Composite_2004_03           READY        EXPORT_IMAGE 2025-10-06 15:58:52.052000
Landsat_Composite_2004_02           READY        EXPORT_IMAGE 2025-10-06 15:58:51.774000
Landsat_Composite_2003_12           READY        EXPORT_IMAGE 2025-10-06 15:58:51.447000
Landsat_Composite_200

Landsat_Composite_1995_07           READY        EXPORT_IMAGE 2025-10-06 15:58:20.740000
Landsat_Composite_1995_06           READY        EXPORT_IMAGE 2025-10-06 15:58:20.344000
Landsat_Composite_1995_05           READY        EXPORT_IMAGE 2025-10-06 15:58:19.975000
Landsat_Composite_1995_03           READY        EXPORT_IMAGE 2025-10-06 15:58:19.533000
Landsat_Composite_1995_02           READY        EXPORT_IMAGE 2025-10-06 15:58:19.170000
Landsat_Composite_1995_01           READY        EXPORT_IMAGE 2025-10-06 15:58:18.643000
Landsat_Composite_1994_12           READY        EXPORT_IMAGE 2025-10-06 15:58:18.297000
Landsat_Composite_1994_11           READY        EXPORT_IMAGE 2025-10-06 15:58:17.575000
Landsat_Composite_1994_10           READY        EXPORT_IMAGE 2025-10-06 15:58:17.183000
Landsat_Composite_1994_09           READY        EXPORT_IMAGE 2025-10-06 15:58:16.805000
Landsat_Composite_1994_08           READY        EXPORT_IMAGE 2025-10-06 15:58:16.526000
Landsat_Composite_199

...

Landsat_DSWE_2024_10                READY        EXPORT_IMAGE 2025-10-06 15:45:11.401000
Landsat_DSWE_2024_09                READY        EXPORT_IMAGE 2025-10-06 15:45:11.128000
Landsat_DSWE_2024_08                READY        EXPORT_IMAGE 2025-10-06 15:45:10.742000
Landsat_DSWE_2024_07                READY        EXPORT_IMAGE 2025-10-06 15:45:10.404000
Landsat_DSWE_2024_06                READY        EXPORT_IMAGE 2025-10-06 15:45:10.060000
Landsat_DSWE_2024_05                READY        EXPORT_IMAGE 2025-10-06 15:45:09.661000
Landsat_DSWE_2024_04                READY        EXPORT_IMAGE 2025-10-06 15:45:09.363000
Landsat_DSWE_2024_03                READY        EXPORT_IMAGE 2025-10-06 15:45:09.047000
Landsat_DSWE_2024_02                READY        EXPORT_IMAGE 2025-10-06 15:45:08.620000
Landsat_DSWE_2024_01                READY        EXPORT_IMAGE 2025-10-06 15:45:08.244000
Landsat_DSWE_2023_12                READY        EXPORT_IMAGE 2025-10-06 15:45:07.899000
Landsat_DSWE_2023_11 

Landsat_DSWE_2017_01                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:39.657000
Landsat_DSWE_2016_12                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:39.326000
Landsat_DSWE_2016_11                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:39.036000
Landsat_DSWE_2016_10                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:38.615000
Landsat_DSWE_2016_09                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:38.258000
Landsat_DSWE_2016_08                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:37.954000
Landsat_DSWE_2016_07                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:37.632000
Landsat_DSWE_2016_06                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:37.317000
Landsat_DSWE_2016_05                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:36.986000
Landsat_DSWE_2016_04                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:36.717000
Landsat_DSWE_2016_03                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:36.417000
Landsat_DSWE_2016_02 

Landsat_DSWE_2005_03                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:06.342000
Landsat_DSWE_2005_02                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:05.738000
Landsat_DSWE_2004_09                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:05.368000
Landsat_DSWE_2004_08                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:04.918000
Landsat_DSWE_2004_07                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:04.455000
Landsat_DSWE_2004_06                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:03.861000
Landsat_DSWE_2004_05                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:03.456000
Landsat_DSWE_2004_04                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:03.063000
Landsat_DSWE_2004_03                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:02.429000
Landsat_DSWE_2004_02                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:02.125000
Landsat_DSWE_2003_12                COMPLETED    EXPORT_IMAGE 2025-10-06 15:44:01.773000
Landsat_DSWE_2003_11 

Landsat_DSWE_1995_07                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:31.130000
Landsat_DSWE_1995_06                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:30.749000
Landsat_DSWE_1995_05                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:30.289000
Landsat_DSWE_1995_03                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:29.713000
Landsat_DSWE_1995_02                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:29.235000
Landsat_DSWE_1995_01                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:28.904000
Landsat_DSWE_1994_12                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:28.401000
Landsat_DSWE_1994_11                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:27.874000
Landsat_DSWE_1994_10                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:27.226000
Landsat_DSWE_1994_09                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:26.855000
Landsat_DSWE_1994_08                COMPLETED    EXPORT_IMAGE 2025-10-06 15:43:26.291000
Landsat_DSWE_1994_07 

Landsat_Composite_2025_01           COMPLETED    EXPORT_IMAGE 2025-10-06 12:27:05.853000
Landsat_QC_2025_07                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:35.946000
Landsat_QC_2025_06                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:35.575000
Landsat_QC_2025_05                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:35.177000
Landsat_QC_2025_04                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:34.725000
Landsat_QC_2025_03                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:34.017000
Landsat_QC_2025_02                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:33.582000
Landsat_QC_2025_01                  COMPLETED    EXPORT_IMAGE 2025-10-06 12:25:32.822000
Landsat_DSWE_2025_07                COMPLETED    EXPORT_IMAGE 2025-10-06 12:23:54.220000
Landsat_DSWE_2025_06                COMPLETED    EXPORT_IMAGE 2025-10-06 12:23:53.574000
Landsat_DSWE_2025_05                COMPLETED    EXPORT_IMAGE 2025-10-06 12:23:52.991000
Landsat_DSWE_2025_04 

# Download .tif files from Drive to local directory

In [2]:
def authenticate_drive():
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile("client_secrets.json")
    gauth.LocalWebserverAuth()  # Opens a browser for authentication
    return GoogleDrive(gauth)

def find_all_landsat_dswe_folders(drive):
    query = f"title='{TARGET_FOLDER_NAME}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    return drive.ListFile({'q': query}).GetList()

def list_tif_files_in_folder(drive, folder_id):
    query = f"'{folder_id}' in parents and trashed=false and mimeType != 'application/vnd.google-apps.folder'"
    return [
        f for f in drive.ListFile({'q': query}).GetList()
        if any(f['title'].lower().endswith(ext) for ext in TIF_EXTENSIONS)
    ]

def download_files(files, download_dir):
    os.makedirs(download_dir, exist_ok=True)
    for file in files:
        file_path = os.path.join(download_dir, file['title'])
        print(f"Downloading: {file['title']} -> {file_path}")
        file.GetContentFile(file_path)

def main():
    drive = authenticate_drive()
    folders = find_all_landsat_dswe_folders(drive)
    all_tif_files = []

    for folder in folders:
        print(f"Found folder: {folder['title']} (ID: {folder['id']})")
        tif_files = list_tif_files_in_folder(drive, folder['id'])
        print(f" - {len(tif_files)} .tif files found")
        all_tif_files.extend(tif_files)

    download_files(all_tif_files, DOWNLOAD_DIR)
    print(f"\n✅ Download complete! All files are in: {DOWNLOAD_DIR}")

In [4]:
TARGET_FOLDER_NAME = "Landsat_QC"
DOWNLOAD_DIR = r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks"
TIF_EXTENSIONS = [".tif", ".tiff"]

if __name__ == "__main__":
    main()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=153793833549-rnci8u36klc20qro4ccpd2e44i0lagne.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.
Found folder: Landsat_QC (ID: 1U1fLExB06VGg2fE6fkD6CZT7jYXA0hFI)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1t3C6vOmxwMM-zaBO5J0qfzDfWnbKffsz)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1cWc8Fu5XG7Ke3IENmloCL0io2on8xTcL)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 17HPs_QHwT5g5onj0M2LXFiiZyYZXQs7B)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1TXfFL3fAur4_xjfPS4XqzFXdQhj4cvzV)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1WEwSKHwd_Nr3u4l4JiX5H-ClvwvWHbiJ)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1i-sHA4wyBMtvY2jef75SyTedeqXoyPP4)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1sBBRVWTzysblVP2SzhUB4BkVz

 - 1 .tif files found
Found folder: Landsat_QC (ID: 1IYG_nb_5Flatg7wZlS4PEnGU8IDsNBmy)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1svMftlLwwyJzeL9K7lYy2v2Lxr1YEKaV)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1LU9HQuCzTdZU2BuHxwTsR8nMJXElxSB8)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1sGBSvjqNKvu57_GLPz3dxvEKkcYjlSw0)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1ggvgC7XxVEr0NAfiaL3N9KcXlJsdibRG)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1raue-3Gi1nFCfXPqoIV3-of_aWabrVEi)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1StbjVib7EdfQE9HwBk0KrKkhsyNzclcu)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1ppOE69hL_TqfNvJWGVUgN3B65T51lXbv)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1A9lVHeZWe_4Wk6dWct5M9kb3QseGWUYr)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1OBJouBGywSqNP7sHynF8_j4CDqw4qr8k)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1OoPshDS_VzXHwY2QKPEySzDx9Z_eIuOt)
 - 1 .tif files found
Found folder: Landsat

 - 1 .tif files found
Found folder: Landsat_QC (ID: 1eSJFBbc5CSz0jk7ulQ8hoLmvYs3bkOlB)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1q6WfXCTBTjBnw3In7LpLXetwVc7pnQWI)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1U7IkCkkJfgUtfkAFg1R1jYp8J61BMrWG)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1l2Cs5SwCpIf1ydnD-DLJdLPJvqGypWtO)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1JUERx-PM1wV5GZwEiC9Yz1gC-3bhlpCr)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1fw-Qoh7I65RSEkBtOWwFTO3CN9d71Uz6)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 11sZfcKHP-zcFQw4CoyC9cZLvDGir9CIb)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1AxmkoWSj3mkse5jd7grpFbA6NBJohnJS)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1jwOCdV0HLcuT4yfcGhD4q8t66xFOgyDH)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1SsBg9t6pQooFt-AHbAyQsC81wx0wssYt)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1HCjDh5JBJ-ow37hbJ3FoPPR4nZtms4Tn)
 - 1 .tif files found
Found folder: Landsat

 - 1 .tif files found
Found folder: Landsat_QC (ID: 1LxyusGmBDerIFgmk94uLP9-u04NNvXZy)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 18Frg2SyzDdZeX31d7-k-c4zV_EYcyjYv)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1x_MrgNzruDqod-ZvxCIjrQKVzRNmSx23)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1iO6p2IHiYx4QAqnsMwgh4t7t8oS5bF9Z)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1cw7dW0gNtsT5-PlhFvsuljDiEelpAczB)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1mOCO0_HXZknIHJ5tH71bxgQz0KW7uuJo)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1tsMywNMoXswb3TJqBTrAKeWwnOi9EsIu)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1YJHT4ZUihRXN0ljOvhMiBTroQmDjR3Kw)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1RVtdG0fIxZhStcTpcrzfPgQ2W9-38aMr)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1Y9GLylU8UeEbJpKtIsVTxF8lWYRJqRV3)
 - 1 .tif files found
Found folder: Landsat_QC (ID: 1ERdOqLiZ6RLuhGDUMKhTYcAroIAj4eOu)
 - 1 .tif files found
Found folder: Landsat

 - 1 .tif files found
Downloading: QC_2025_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_07.tif
Downloading: QC_2025_06.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_06.tif
Downloading: QC_2025_05.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_05.tif
Downloading: QC_2025_04.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_04.tif
Downloading: QC_2025_03.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_03.tif
Downloading: QC_2025_02.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_02.tif
Downloading: QC_2025_01.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2025_01.tif
Downloading: QC_2024_12.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Water

Downloading: QC_2020_02.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2020_02.tif
Downloading: QC_2020_01.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2020_01.tif
Downloading: QC_2019_12.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2019_12.tif
Downloading: QC_2019_11.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2019_11.tif
Downloading: QC_2019_10.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2019_10.tif
Downloading: QC_2019_09.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2019_09.tif
Downloading: QC_2019_08.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2019_08.tif
Downloading: QC_2019_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks

Downloading: QC_2014_09.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_09.tif
Downloading: QC_2014_08.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_08.tif
Downloading: QC_2014_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_07.tif
Downloading: QC_2014_06.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_06.tif
Downloading: QC_2014_05.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_05.tif
Downloading: QC_2014_04.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_04.tif
Downloading: QC_2014_03.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2014_03.tif
Downloading: QC_2014_02.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks

Downloading: QC_2005_04.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2005_04.tif
Downloading: QC_2005_03.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2005_03.tif
Downloading: QC_2005_02.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2005_02.tif
Downloading: QC_2004_09.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2004_09.tif
Downloading: QC_2004_08.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2004_08.tif
Downloading: QC_2004_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2004_07.tif
Downloading: QC_2004_06.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_2004_06.tif
Downloading: QC_2004_05.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks

Downloading: QC_1998_08.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_08.tif
Downloading: QC_1998_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_07.tif
Downloading: QC_1998_06.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_06.tif
Downloading: QC_1998_05.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_05.tif
Downloading: QC_1998_04.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_04.tif
Downloading: QC_1998_03.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_03.tif
Downloading: QC_1998_02.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1998_02.tif
Downloading: QC_1998_01.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks

Downloading: QC_1992_01.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1992_01.tif
Downloading: QC_1991_11.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_11.tif
Downloading: QC_1991_10.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_10.tif
Downloading: QC_1991_09.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_09.tif
Downloading: QC_1991_08.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_08.tif
Downloading: QC_1991_07.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_07.tif
Downloading: QC_1991_06.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks\QC_1991_06.tif
Downloading: QC_1991_05.tif -> C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\WaterMasks\Landsat\QC_Masks